In [ ]:
import pandas as pd
from joblib import dump, load

from xgboost import XGBClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
# общие настройки
RANDOM_STATE = 7
SEARCH_PARAMS = {
    'scoring': 'roc_auc',
    'refit'  : True,
    'n_iter' : 50,              
    'n_jobs' : -1, 
    'verbose': 1,
    'cv'     : 3
}


# PROCESS DATA

In [ ]:
df = pd.read_csv('data/train.csv',index_col='id')
test = pd.read_csv('data/test.csv',index_col='id')


In [ ]:
X = df.drop('cardio',axis=1)
y = df.cardio


In [ ]:
# приготовим два тестовых признака
def test_feature(X_):
    X_ = X_.copy()
    X_['test_feature']   = X_['ap_hi'] * X_['ap_lo']
    X_['test_feature_2'] = X_['gluc']  * X_['cholesterol']
    
    return X_

In [ ]:
# вся предобработка вынесена в отдельный файл
from custom_transformers import set_buckets, BMITransformer, APTransformer

# коррекция выбросов, заполнение пропущенных значений
# добавление экспериментальных признаков
clean_the_data = make_pipeline(
    FunctionTransformer(set_buckets),
    BMITransformer(),
    APTransformer(),
    FunctionTransformer(test_feature),
)

# кодирование и масштабирование
to_scale  = ['age',
             'bmi',
             'weight_bucket',
             'ap_hi',
             'ap_lo',
             'test_feature',
             'test_feature_2',
            ]

to_encode = ['gender']

to_pass = ['smoke',
           'alco',
           'active',
           'cholesterol',
           'gluc',
          ]

transformers = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), to_encode),
    ('num', StandardScaler(), to_scale),
    ('pss', 'passthrough', to_pass)
])

# общий пайплайн с предобработкой
preprocess = make_pipeline(
    clean_the_data,
    transformers
)    


In [ ]:
# функция вывода важности признаков
def show_feature_importances(model):
    all_names = model[0][-1].get_feature_names_out()
    display(pd.DataFrame({'features':all_names,
                          'feature_importances':model._final_estimator.feature_importances_}) \
              .sort_values(by='feature_importances',ascending=False))


# XGBOOST

In [ ]:
%%time

# пайплайн модели
pipe_xgb = Pipeline(steps=[
    ('pre', preprocess),
    ('xgb', XGBClassifier(random_state=RANDOM_STATE))
])    

# гиперпараметры для оптимизации
xgb_params = {
    'xgb__learning_rate': [0.01, 0.05, 0.1, 0.25, 0.5],
    'xgb__max_depth': [3, 5, 8, 10, 12, 16],
    'xgb__n_estimators': range(200, 601, 20),
    'xgb__gamma': [0, 0.1, 0.5],
    'xgb__subsample': [0.2, 0.4, 0.6, 0.8, 1],
    'xgb__colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1],
    'xgb__reg_alpha': [0, 1, 2, 3, 4, 5, 10],
    'xgb__reg_lambda': [0, 1, 2, 3, 4, 5, 10]
}

# поиск оптимальной модели по roc_auc
xgb = RandomizedSearchCV(pipe_xgb,
                         xgb_params,
                         **SEARCH_PARAMS)

xgb.fit(X,y)

# вывод лучшей метрики и параметров
display(xgb.best_score_)
display(xgb.best_params_)

xgb_best = xgb.best_estimator_

# вывод важности признаков
show_feature_importances(xgb_best)


# RANDOM FOREST

In [ ]:
%%time

# пайплайн модели
pipe_rfc = Pipeline(steps=[
    ('pre',preprocess),
    ('rfc', RandomForestClassifier(random_state=RANDOM_STATE))
])

rfc_params = {'rfc__n_estimators'     : range(200,601,20),
              'rfc__min_samples_split': range(50,101,5),
              'rfc__min_samples_leaf' : range(20,41,4),
              'rfc__max_depth'        : [8,10,12,14,16,20,24],
              } 
    
rfc = RandomizedSearchCV(pipe_rfc,
                         rfc_params,
                         **SEARCH_PARAMS)

rfc.fit(X,y)

# вывод лучшей метрики и параметров
display(rfc.best_score_)
display(rfc.best_params_)

rfc_best = rfc.best_estimator_

# вывод важности признаков
show_feature_importances(rfc_best)

# RESULTS

In [ ]:
# экспорт результатов – модели, предобработки и предсказания

def export_result(model):
    
    predict_proba = model.predict_proba(test)[:,1]

    model_name = type(model[-1]).__name__

    pd.DataFrame(index=test.index,
                 data=predict_proba,
                 columns=['cardio']) \
      .to_csv(model_name + '_test_predict_proba.csv')
        
    '''
    Коррекция признаков, поиск ошибок, выбросы, заполнение пропусков 
    – эти процедуры актуальны для датасетов, например для архивных данных.
    
    Для интерактивного приложения с предсказанием 
    по единственному набору – можно ограничить предобработку 
    стандартным масштабированием и кодированием.
    
    Поэтому выгружаем избранное )
    
    1. кодирование-масштабирование
    2. модель отдельно
    3. полная версия пайплайна с предобработкой и моделью
    '''
    
    dump(model[0][-1], model_name + '_scaler_encoder.joblib') 
    dump(model[-1], model_name + '_model.joblib') 
    dump(model, model_name + '_full_pipeline.joblib')


In [ ]:
export_result(xgb_best)
export_result(rfc_best)
